# 인공지능 작사가 만들기
----------

## Step 1. 데이터 다운로드

```python
$ wget https://aiffelstaticprd.blob.core.windows.net/media/documents/song_lyrics.zip
$ unzip song_lyrics.zip -d ~/aiffel/lyricist/data/lyrics  #lyrics 폴더에 압축풀기
```

## Step 2. 데이터 읽어오기

In [4]:
import glob
import os

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['Load up on guns, bring your friends', "It's fun to lose and to pretend", "She's over-bored and self-assured"]


## Step 3. 데이터 정제

In [5]:
import re                  # 정규표현식을 위한 Regex 지원 모듈 (문장 데이터를 정돈하기 위해) 
import numpy as np         # 변환된 문장 데이터(행렬)을 편하게 처리하기 위해
import tensorflow as tf    # 대망의 텐서플로우!
import os


def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()       # 소문자로 바꾸고 양쪽 공백을 삭제
  
    # 아래 3단계를 거쳐 sentence는 스페이스 1개를 delimeter로 하는 소문자 단어 시퀀스로 바뀝니다.
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)        # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)                  # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환

    sentence = sentence.strip()

    sentence = '<start> ' + sentence + ' <end>'      # 이전 스텝에서 본 것처럼 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 줍니다
    
    return sentence

print(preprocess_sentence("This @_is ;;;sample        sentence."))   # 이 문장이 어떻게 필터링되는지 확인해 보세요.

<start> this is sample sentence . <end>


In [6]:
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
        
    corpus.append(preprocess_sentence(sentence))
        
corpus[:10]

['<start> load up on guns , bring your friends <end>',
 '<start> it s fun to lose and to pretend <end>',
 '<start> she s over bored and self assured <end>',
 '<start> oh no , i know a dirty word hello , hello , hello , how low <end>',
 '<start> hello , hello , hello , how low <end>',
 '<start> hello , hello , hello , how low <end>',
 '<start> hello , hello , hello with the lights out , it s less dangerous <end>',
 '<start> here we are now , entertain us <end>',
 '<start> i feel stupid and contagious <end>',
 '<start> here we are now , entertain us <end>']

In [11]:
def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000,  # 전체 단어의 개수 
        filters=' ',    # 별도로 전처리 로직을 추가할 수 있습니다. 이번에는 사용하지 않겠습니다.
        oov_token="<unk>"  # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지
    )
    tokenizer.fit_on_texts(corpus)   # 우리가 구축한 corpus로부터 Tokenizer가 사전을 자동구축하게 됩니다.

    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축하게 됩니다.
    tensor = tokenizer.texts_to_sequences(corpus)   # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환합니다.

    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding  메소드를 제공합니다.
    # maxlen의 디폴트값은 None입니다. 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰집니다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=15)  

    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2 2347   29 ...    0    0    0]
 [   2   11   16 ...    0    0    0]
 [   2   48   16 ...    0    0    0]
 ...
 [   2    9  150 ...    0    0    0]
 [   2    8    9 ...    0    0    0]
 [   2    6 3239 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f1546845f90>


In [12]:
src_input = tensor[:, :-1]  # tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다. 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
tgt_input = tensor[:, 1:]    # tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.

BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1    # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

## Step 4. 평가 데이터셋 분리

In [13]:
from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, 
                                                                tgt_input, 
                                                                test_size=0.2,
                                                                shuffle=True)

In [14]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (140599, 14)
Target Train: (140599, 14)


## Step 5. 인공지능 만들기

In [16]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [17]:
for src_sample, tgt_sample in dataset.take(1): break
model(src_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[-7.2590018e-07,  5.5863009e-05,  3.2944918e-05, ...,
         -2.6253267e-04, -4.9867592e-04, -1.5287944e-04],
        [-2.0553951e-05,  5.0189898e-05,  7.6599419e-05, ...,
         -4.9186649e-04, -5.6189951e-04, -1.1026534e-04],
        [ 2.6264859e-04,  2.6532135e-04, -9.6551264e-07, ...,
         -3.8864498e-04, -5.9992331e-04, -1.2942370e-04],
        ...,
        [ 1.6138203e-05, -4.5740348e-04, -9.8037347e-04, ...,
          4.6476186e-04, -2.0236234e-04, -2.0494222e-04],
        [ 1.5961687e-04, -8.1279711e-04, -7.5418164e-04, ...,
          8.1015105e-04, -1.3280784e-04,  5.2590269e-05],
        [ 9.3774150e-05, -1.0591306e-03, -3.9921596e-04, ...,
          1.1775694e-03,  1.3240543e-04,  2.1510979e-04]],

       [[-7.2590018e-07,  5.5863009e-05,  3.2944918e-05, ...,
         -2.6253267e-04, -4.9867592e-04, -1.5287944e-04],
        [ 7.0765673e-05, -1.3177624e-04,  8.8938228e-05, ...,
         -2.0049297e-04, 

In [18]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  3072256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  12301025  
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


In [19]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10)

Epoch 1/10
686/686 [==============================] - 107s 156ms/step - loss: 3.6037
Epoch 2/10
686/686 [==============================] - 107s 156ms/step - loss: 3.1332
Epoch 3/10
686/686 [==============================] - 110s 160ms/step - loss: 2.9450
Epoch 4/10
686/686 [==============================] - 109s 158ms/step - loss: 2.7980
Epoch 5/10
686/686 [==============================] - 109s 159ms/step - loss: 2.6694
Epoch 6/10
686/686 [==============================] - 109s 159ms/step - loss: 2.5544
Epoch 7/10
686/686 [==============================] - 109s 159ms/step - loss: 2.4491
Epoch 8/10
686/686 [==============================] - 109s 159ms/step - loss: 2.3504
Epoch 9/10
686/686 [==============================] - 108s 158ms/step - loss: 2.2569
Epoch 10/10
686/686 [==============================] - 109s 158ms/step - loss: 2.1675


## Step 5. 인공지능 만들기

In [20]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다. 

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, 
																 tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <end>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

In [22]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you , i love you <end> '

## 회고

고칠게 많지 않아서 따라가기 수월했고, 재미있었다.     
확실히 뭔가 할수 있구나가 보여야 재미가 붙는 것 같다.     
그리고 진짜 신기한건 정말로 누군가가 써 준것 마냥 뭔가 감정이 있는 느낌의 가사를 작사해준다.    
약간 시적인 느낌이다.     
뭔가 지금까지 이렇게 와닿은 노드는 없었는데 내 길을 찾은건지 조금 더 생각해 봐야겠다.